In [3]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [4]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [31]:
def pipeline_new_scans(scan_paths, label, model):
    x = np.array([helper.process_scan(path) for path in scan_paths])
    y = [label[sc.split("/")[-1].split("_L")[0]] for sc in scan_paths]

    loader = tf.data.Dataset.from_tensor_slices((x, y))
    batch_size = 1

    # Only rescale.
    dataset = (
        loader.shuffle(len(x))
        .map(data_augmentation.validation_preprocessing)
        .batch(batch_size)
        .prefetch(2)
    )
    y_pred = model.predict(dataset)
    mse = tf.keras.losses.MeanAbsoluteError(reduction="auto", name="mean_absolute_error")
    return mse(y, y_pred).numpy()    

In [32]:
# Load best weights.
model = load_model("3d_image_classification_v1.h5")

In [33]:
# Test
test = pd.read_csv('test_cn_images.csv')
test['Image Filename'] = test['Image Filename'].apply(lambda x: x.split(".nii.gz")[0])

test_scan_paths = [
    os.path.join(os.getcwd(), "CN", x)
    for x in os.listdir("CN")
    if "_L" in x
    if x.split("_L")[0] in test['Image Filename'].values
]
print("Test: " + str(len(test_scan_paths)))

label_test = helper.get_age(test_scan_paths, test)


Test: 39


In [37]:
pipeline_new_scans(test_scan_paths, label_test, model)

ResourceExhaustedError:  OOM when allocating tensor with shape[1,64,62,62,62] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node 3dcnn/conv3d_16/Conv3D (defined at <ipython-input-31-fa87f066f00e>:15) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_predict_function_7813]

Function call stack:
predict_function


## AD

In [109]:
ad = pd.read_csv("ad_jonas_209.csv")
ad['Image Filename'] = ad[['Subject ID', 'Visit']].apply(lambda x: x[0] + "_" + x[1].replace(" ", "_").replace("/", ""), axis=1)

ad_scan_paths = [
    os.path.join(os.getcwd(), "AD", x)
    for x in os.listdir("AD")
    if "_L" in x
    if x.split("_L")[0] in ad['Image Filename'].values
]
print("AD: " + str(len(ad_scan_paths)))
label_ad = helper.get_age(ad_scan_paths, ad)
pipeline_new_scans(ad_scan_paths, label_ad)

AD: 120


5.8301797

## MCI

In [110]:
mci = pd.read_csv("mci_jonas_251.csv")
mci['Image Filename'] = mci[['Subject ID', 'Visit']].apply(lambda x: x[0] + "_" + x[1].replace(" ", "_").replace("/", ""), axis=1)

mci_scan_paths = [
    os.path.join(os.getcwd(), "MCI", x)
    for x in os.listdir("MCI")
    if "_L" in x
    if x.split("_L")[0] in mci['Image Filename'].values
]
print("MCI: " + str(len(mci_scan_paths)))

label_mci = helper.get_age(mci_scan_paths, mci)
pipeline_new_scans(mci_scan_paths, label_mci)

MCI: 153


5.0703893